In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

project_root = os.path.abspath('/Users/subhojit/workspace/saturn/src')
if project_root not in sys.path:
    sys.path.append(project_root)
from convnet.data_loader import CIFAR_10_DataLoader
from convnet.struct.layers import Linear, ReLU

%matplotlib inline

In [2]:
file_directory = '/Users/subhojit/Downloads/cifar-10-batches-py'
cdl = CIFAR_10_DataLoader()
xtrain_data, ytrain_data, Xtest, ytest = cdl.load_cifar_10_dataset(file_directory)

xtrain_data = xtrain_data.astype('float32') / 255.0
Xtest = Xtest.astype('float32') / 255.0

# np.random.shuffle(Xtrain)
n1 = int(0.8 * len(xtrain_data))
Xtrain = xtrain_data[:n1]
ytrain = ytrain_data[:n1]
Xdev = xtrain_data[n1:]
ydev = ytrain_data[n1:]

num_classes = len(set(ytrain))

In [4]:
# data preparation
np.random.seed(231)
std_dev = 1e-2
n_hidden = 200

layers = [
    Linear(3072, n_hidden, std_dev=std_dev),                ReLU(),
    Linear(n_hidden, n_hidden, std_dev=std_dev),            ReLU(),
    Linear(n_hidden, n_hidden, std_dev=std_dev),            ReLU(),
    Linear(n_hidden, num_classes, std_dev=std_dev)
]

params = [p for layer in layers for p in layer.parameters()]
# for layer in layers:
#     for p in layer.parameters():
#         params.append(p)

print(sum(par.size for par in params))

697010


In [5]:
def softmax_loss(scores, y):
    epsilon = 1e-12
    num_examples = scores.shape[0]
    shifted_scores = scores - np.max(scores, axis=1, keepdims=True)
    exp_scores = np.exp(shifted_scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    correct_log_probs = -np.log(probs[np.arange(num_examples), y] + epsilon)
    loss = np.mean(correct_log_probs)
    return loss

def softmax_numpy(x, axis=1):
    x_shifted = x - np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

In [ ]:
max_iterations = 1000
batch_size = 128
lossi = []
Hs = []

rev_layers = layers[::-1]

for i in range(max_iterations):

    #mini batch
    ix = np.random.randint(0, Xtrain.shape[0], (batch_size,))
    Xb, Yb = Xtrain[ix], ytrain[ix]

    #farward pass
    x = Xb
    for layer in layers:
        x = layer.forward(x)

    logits = x
    loss = softmax_loss(logits, Yb)
    lossi.append(loss)

    #backward pass
    ddlogits = softmax_numpy(logits)
    ddlogits[np.arange(Xb.shape[0]), Yb] -= 1
    ddlogits /= Xb.shape[0]

    for layer in rev_layers:
        if isinstance(layer, Linear):
            layer.backward(ddlogits)
        elif isinstance(layer, ReLU):
            layer.backward(ddlogits)


    grads = [dW1, db1, dW2, db2]
    # print(grads)
    lr = 0.01
    W1 += -lr * dW1
    b1 += -lr * db1
    W2 += -lr * dW2
    b2 += -lr * db2
    if i % 100 == 0:
        print(f"loss: {loss:.4f}")